# Random Forest Classifier

## Highest Model Score: 92.80%

In [55]:
from data import Database
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load data
db = Database()
# db.reset()
# db.seed(3000)

df = db.dataframe()

# Find Baseline
baseline = df['Rarity'].value_counts(normalize=True).max()

# Create features and target
target = "Rarity"
X = df.drop(columns=[target, 'Name', 'Type', 'Damage', 'Roll', 'Level'])
y = df[target]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_jobs=-1,
                               random_state=42)

model.fit(x_train, y_train)

# Evaluate model
y_pred = np.array(model.predict(x_test))

print(y_test.shape)
print(y_pred.shape)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}\nBaseline: {baseline}")


(600,)
(600,)
Accuracy: 0.9283333333333333
Baseline: 0.30133333333333334


In [56]:
# Tune hyperparameters
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 25, 40, 55, 70, 85, 100],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 25, 40, 55, 70, 85, 100],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=1)

In [57]:
print(grid_search.best_estimator_.score(x_test, y_test))
best_params = grid_search.best_params_
print(best_params)

0.9333333333333333
{'max_depth': 25, 'min_samples_split': 2, 'n_estimators': 200}


In [42]:
from Fortuna import random_float, random_int
from MonsterLab.monster_data import Random
import pandas as pd
from damage_parser import parse_damage

rand = Random()
# Retrain the model with the best_params.
model_trained = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                       max_depth=best_params['max_depth'],
                                       min_samples_split=best_params['min_samples_split'],
                                       n_jobs=-1,
                                       random_state=42)

model_trained.fit(x_train, y_train)

options = ["Health", "Energy", "Sanity", "Low", "High", "Rarity"]
stats = [round(random_float(1, 250), 2) for _ in range(3)]
level = random_int(1, 84)
health = stats.pop()
energy = stats.pop()
sanity = stats.pop()
damage = f"{level}d{rand.dice[rand.random_rank()]}{rand.bonus()}"
low, high, _ = parse_damage(damage)

data = pd.DataFrame([dict(zip(options, (health, energy, sanity, low, high)))])

prediction = model.predict(data)

proba_pred = model.predict_proba(data).max()

In [119]:
y_pred = np.array(model.predict(x_test))

print(f"Accuracy: {accuracy_score(y_test, y_pred)}\nBaseline: {baseline}")

Accuracy: 0.9283333333333333
Baseline: 0.30133333333333334


In [70]:
print(f"Prediction: Rank {prediction[0]}\nConfidence: {proba_pred*100}%")

Prediction: Rank 1
Confidence: 40.0%


In [76]:
x_train.shape, y_train.shape

((2400, 5), (2400,))

# Neural Network

## Highest Model Score: 94.83%

In [114]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

tf.keras.backend.clear_session()

model_tf = Sequential([
    Dense(10, activation='relu'),
    Dense(20, activation='relu'),
    Dense(12, activation='relu'),
    Dense(6, activation='softmax')
])

callback = EarlyStopping(monitor='val_loss', patience=3)

model_tf.compile(loss='sparse_categorical_crossentropy',
                 optimizer=Adam(learning_rate=0.001),
                 metrics=['accuracy'])

history = model_tf.fit(x_train, y_train, batch_size=200,
                       epochs=200,
                       callbacks=[callback],
                       validation_data=(x_test, y_test))


Epoch 1/200
12/12 [==============================] - 2s 23ms/step - loss: 3.6750 - accuracy: 0.1646 - val_loss: 2.6270 - val_accuracy: 0.1833
Epoch 2/200
12/12 [==============================] - 0s 6ms/step - loss: 2.1351 - accuracy: 0.2179 - val_loss: 1.8131 - val_accuracy: 0.3083
Epoch 3/200
12/12 [==============================] - 0s 7ms/step - loss: 1.6039 - accuracy: 0.4750 - val_loss: 1.5055 - val_accuracy: 0.4917
Epoch 4/200
12/12 [==============================] - 0s 7ms/step - loss: 1.4325 - accuracy: 0.5350 - val_loss: 1.3995 - val_accuracy: 0.5417
Epoch 5/200
12/12 [==============================] - 0s 6ms/step - loss: 1.3844 - accuracy: 0.5421 - val_loss: 1.3731 - val_accuracy: 0.5283
Epoch 6/200
12/12 [==============================] - 0s 7ms/step - loss: 1.3670 - accuracy: 0.5462 - val_loss: 1.3624 - val_accuracy: 0.5617
Epoch 7/200
12/12 [==============================] - 0s 18ms/step - loss: 1.3539 - accuracy: 0.5767 - val_loss: 1.3498 - val_accuracy: 0.5750
Epoch 8/200

In [115]:
accuracy = max(history.history['accuracy'])
val_accuracy = max(history.history['val_accuracy'])
print(f"Accuracy: {accuracy}\nValidation Accuracy: {val_accuracy}")

Accuracy: 0.949999988079071
Validation Accuracy: 0.9549999833106995


In [117]:
loss, accuracy = model_tf.evaluate(x_test, y_test, batch_size=50)

12/12 [==============================] - 0s 3ms/step - loss: 0.1567 - accuracy: 0.9483


In [118]:
print(f"Evaluation Accuracy: {accuracy}\nEvaluation Loss: {loss}")

Evaluation Accuracy: 0.9483333230018616
Evaluation Loss: 0.15672776103019714
